In [1]:
from model import DeepFM3Times

In [2]:
import torch
import torch.nn as nn

In [3]:
import pickle as pk

In [4]:
data = pk.load(open('./data_train.pk', 'rb'))

In [5]:
data = torch.LongTensor(data).cuda()

In [6]:
num_of_users = torch.max(data[:, 0]).item() + 1
num_of_times = 7344
num_of_locs = torch.max(data[:, 2]).item() + 1
num_of_tofd = 48
num_of_days = num_of_times // num_of_tofd

In [7]:
deepfm = DeepFM3Times(num_of_users, num_of_times, num_of_tofd, num_of_days, num_of_locs, rank=2, latent_dim=8).cuda()

In [8]:
optimizer = torch.optim.Adam(deepfm.parameters(), lr=1e-3)
optimizer_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [9]:
data_test = pk.load(open('./data_test.pk', 'rb'))
data_test = torch.LongTensor(data_test).cuda()

In [10]:
data_size = data.shape[0]
batch_size = 64

for it in range(1, 101):
    avg_loss = 0.0
    optimizer_scheduler.step()
    optimizer.zero_grad()
    data = data[torch.randperm(data_size)]
    
    for i in range(0, data_size, batch_size):
        x_u = data[i: i + batch_size, 0]
        x_dt = data[i: i + batch_size, 1]
        x_t = x_dt % num_of_tofd
        x_d = x_dt // num_of_tofd
        y_l = data[i: i + batch_size, 2]
        loss = deepfm(x_u, x_dt, x_d, x_t, y_l)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        avg_loss += loss.item()
        print('Iteration {:04d}, {:.1f}% avg_loss={:.4f}'.format(it, 100.0 * (i + batch_size) / data_size, avg_loss / (i + batch_size)), end='\r')
        
    avg_loss = 0.0

    with torch.no_grad():
        for i in range(0, data_test.shape[0], 256):
            x_u = data_test[i: i + batch_size, 0]
            x_dt = data_test[i: i + batch_size, 1]
            x_t = x_dt % num_of_tofd
            x_d = x_dt // num_of_tofd
            y_l = data_test[i: i + batch_size, 2]
            avg_loss += deepfm(x_u, x_dt, x_d, x_t, y_l).item()
            
    print()
    print('val_avg_loss = {:.4}'.format(avg_loss / data_test.shape[0]))

Iteration 0001, 100.0% avg_loss=6.5505
val_avg_loss = 1.396
Iteration 0002, 100.0% avg_loss=4.3091
val_avg_loss = 0.8654
Iteration 0003, 100.0% avg_loss=2.8182
val_avg_loss = 0.6672
Iteration 0004, 100.0% avg_loss=2.2695
val_avg_loss = 0.5893
Iteration 0005, 100.0% avg_loss=2.0295
val_avg_loss = 0.5533
Iteration 0006, 100.0% avg_loss=1.8176
val_avg_loss = 0.5329
Iteration 0007, 100.0% avg_loss=1.7525
val_avg_loss = 0.526
Iteration 0008, 100.0% avg_loss=1.7028
val_avg_loss = 0.5228
Iteration 0009, 100.0% avg_loss=1.6611
val_avg_loss = 0.5209
Iteration 0010, 100.0% avg_loss=1.6245
val_avg_loss = 0.5182
Iteration 0011, 100.0% avg_loss=1.5444
val_avg_loss = 0.5129
Iteration 0012, 100.0% avg_loss=1.5223
val_avg_loss = 0.5131
Iteration 0013, 100.0% avg_loss=1.5050
val_avg_loss = 0.5125
Iteration 0014, 100.0% avg_loss=1.4894
val_avg_loss = 0.5145


KeyboardInterrupt: 

In [ ]:
torch.save(deepfm, 'deepfm_rank{}_latent{}.pytorch'.format(16, 64))

In [ ]:
data_np = data.cpu().numpy()

In [ ]:
import numpy as np

In [ ]:
test_data_u = torch.LongTensor([0] * 1000).cuda()
test_data_t = torch.LongTensor(range(1000)).cuda()

In [ ]:
with torch.no_grad():
    prob = deepfm.predict_prob(test_data_u, test_data_t).cpu().numpy()

In [ ]:
prob.shape

In [ ]:
max_locs = np.argmax(prob, axis=1)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(prob[:, :4])

In [ ]:
max_locs

In [ ]:
data_np[np.where(data_np[:, 0] == 0)]